----
### Jan 22, 2020 part 8-4 Reviewers' product level diversity Index with Multicore and Dask Demo Code for the public.
----
* Name: Jikhan Jeong
* I have tried to measure consumer preference by using their digital foot-prient and review text by using econometrics and deep learning (=NLP)
* This is a simple demo code to get one of the variables from structured data. Total variables is around 200 variables for structured data
* However review data is text data so that in terms of NLP, the matrix for review text can be sparse, dense, and context-dense.
* This is demo code for showing how it works so that I just show simple example. (This is the one of the simplest computation)
* $\bigstar$ due to the time consumption compared to necessary, even though coding is ok, I stop it in here.
* This file will just be used as a smaple code for the general.
* The full code of origianl independent JMP will be uploaded before JMC to prove the validity of process and data, academic intergrity, and originality of ideas 
----

In [1]:
!pwd

/data/cahnrs/jikhan.jeong/amazon


---
# Pre-Defined Functions
---

In [9]:
import pandas as pd
import numpy as np

In [2]:
# sorting by review time and reindex : for review_meta_in_pro_st_only: the first review of reviewers wrote a review on programmable thermostats without missing values in brand and price.
def sort_by_time_reindex(df):
    df.sort_values(by=['reviewTime'], inplace = True)
    df = df.reset_index(drop=True)
    return df

In [3]:
# summary of dataframe
def sum_df(df):
    print('type', type(df))
    print('shape', df.shape)

In [4]:
# sorting by reviewer, reviewTime, and reindex | df = all_the_review_meta_frm_pt_reviewer
def sort_by_reviewer_time_reindex(df):
    df.sort_values(by=['reviewerID','reviewTime'], inplace = True)
    df = df.reset_index(drop=True)
    return df

In [5]:
# The sum of reviews in each unique asin by user and times unitl t-1 | diversity index, among 124368 unqiue asin 
# df = cleaned_sub_cate_price_all
# user = A2VIFZ0H7XCKP1, time = we can sum of t-1  '2005-10-12'   , sub_cate_list(pre-defined)
# Sharnon Index: Ref: https://en.wikipedia.org/wiki/Diversity_index

def sub_asin_sum_and_diversity_by_user_time(df, user, time): # df =cleaned_sub_cate_price_all, user =, time=, cate_list = u_asin_list (= unique_value_list(cleaned_sub_cate_price_all,'asin'))
    
    df = sort_by_reviewer_time_reindex(df)
    u_asin_list = unique_value_list(df,'asin') # 124368 unique list 
    
    em_df = pd.DataFrame({'reviewerID': [user],'reviewTime':[time]})
    
    n=0
    
    for i in u_asin_list:
        
        em_df['sum_{}'.format(i)] =  len(df[(df['reviewerID'] == user) & (df['reviewTime'] < time) & (df['asin'] == i)])
        
        if n% 1000 ==0:
            print('first loop', n)
        n+=1
        
    total = 0
    
    for i in range(2, em_df.shape[1]): #shape[1] column
        total += em_df.iloc[:,i]
        if i% 1000 ==0:
            print('second loop', i)
        
    p=0.0
    h=0.0
    
    for i in range(2, em_df.shape[1]):
        p = float(em_df.iloc[:,i]/total)
        if  p == 0.0:
             h += 0
        elif p > 0.0: 
             h += -1*p*np.log(p) # add +1 for -inf case, log is natural logrithm
                
    em_df['asin_diversity'] = h  # this is the category diversity index for reviewer at time t considering t-1 reviews  
    em_df['cum_review_t-1'] = total # cumulative review of reviewer i until t-1 in all categories 
    sum_df(em_df)
    return em_df
# eg. test = sub_asin_sum_and_diversity_by_user_time(cleaned_sub_cate_price_all,'A0191512Q2Z9IPUAE2RZ', '2014-06-26')


In [6]:
# (Need to do) Dic {reivwer : time dictionary from dataframe} from using review_meta_in_pro_st_onl
# eg. dic_reviewer_time = review_meta_in_pro_st_only.set_index('reviewerID')['reviewTime'].to_dict()
def dictionary_for_reviewer_time(df): #df = review_meta_in_pro_st_only
    dic = df.set_index('reviewerID')['reviewTime'].to_dict()
    return dic
# eg. dic_reviewer_time = dictionary_for_reviewer_time(review_meta_in_pro_st_only)

# reviewer_time_df = review_meta_in_pro_st_only, sub_category_all_df = sub_cate_all, sub_category_list = sub_cate_list

In [7]:
# list of uniqule value in column value in pd.dataframe
def unique_value_list(df, var):
    u_list = df[var].unique()
    u_list = sorted(u_list)
    return u_list 
# Eg. unique_value_list(cleaned_sub_cate_price_rate_asin_anlz_brand,'asin') 

In [8]:
# (Need to do)
def concate_df_and_asin_analysis(reviewer_time_df, sub_category_all_df):
    n=0
    em_list =[]
    dic_reviewer_time  = dictionary_for_reviewer_time(reviewer_time_df)
    for i, time in dic_reviewer_time.items():
        print(i)
        new_df =  sub_cate_sum_and_diversity_by_user_time(sub_category_all_df, i, time)
        em_list.append(new_df)
       
        if n% 1000 ==0:
            print(n)
        n+=1
    asin_analysis_df = pd.concat(em_list)
    # df = pd.merge(reviewer_time_df,cate_analysis_df, how='left', on='reviewerID')
    return asin_analysis_df    
# asin_analysis_df = concate_df_and_cate_analysis(cleaned_sub_cate_price_rate_asin_anlz_brand_pt, cleaned_sub_cate_price_all)

---
# Loading Data: Pandas
---

In [10]:
cleaned_sub_cate_price_all = pd.read_csv('jan_18_2020_cleaned_sub_cate_price_all', index_col=[0])
sum_df(cleaned_sub_cate_price_all)

type <class 'pandas.core.frame.DataFrame'>
shape (185514, 55)


In [11]:
cleaned_sub_cate_price_rate_asin_anlz_brand_pt = pd.read_csv('01_21_2020_cleaned_sub_cate_price_rate_asin_anlz_brand_pt.csv', index_col=[0])
sum_df(cleaned_sub_cate_price_rate_asin_anlz_brand_pt)

type <class 'pandas.core.frame.DataFrame'>
shape (6724, 198)


---
# Single Reviewer Example Demo: 3 hours
* It must be efficiently coded for heavy computation
---

In [ ]:
# Demo for just one reviewer (1, 124372)
%%time
test = sub_asin_sum_and_diversity_by_user_time(cleaned_review_small ,'A2VIFZ0H7XCKP1', '2005-10-12') # 124,368
# need to check the last column values: df.iloc[:-1]

---
# <font color ='blue'> Part 1: Multicore-Processing with pandas dataframe
* Ref: https://stackoverflow.com/questions/45545110/how-do-you-parallelize-apply-on-pandas-dataframes-making-use-of-all-cores-on-o
* Ref:https://www.machinelearningplus.com/python/parallel-processing-python/
* Ref:http://blog.adeel.io/2016/11/06/parallelize-pandas-map-or-apply/ (def parallelize(data,func)) 
---

In [62]:
import multiprocessing as mp
from multiprocessing import cpu_count
print("Number of processors:", mp.cpu_count())

Number of processors: 24


In [65]:
cores = cpu_count() #Number of CPU cores on your system
partitions = cores
partitions

24

In [64]:
def parallelize(data, func):
    data_split = np.array_split(data, partitions)
    pool = mp.Pool(mp.cpu_count())
    data = pd.concat(pool.map(func, data_split),  ignore_index=False)
    pool.close()
    pool.join()
    return data
# df = parallelize(df, func)

In [16]:
dic_reviewer_time = dictionary_for_reviewer_time(cleaned_sub_cate_price_rate_asin_anlz_brand_pt)
print('cleaned reviewers:', len(dic_reviewer_time))

cleaned reviewers: 6724


In [17]:
cleaned_sub_cate_price_all = sort_by_reviewer_time_reindex(cleaned_sub_cate_price_all)
sum_df(cleaned_sub_cate_price_all)

type <class 'pandas.core.frame.DataFrame'>
shape (185514, 55)


In [18]:
u_asin_list = unique_value_list(cleaned_sub_cate_price_all,'asin') 
print('The number of unique product in cleaned 6724 reviewers: ', len(u_asin_list))

The number of unique product in cleaned 6724 reviewers:  124368


In [20]:
clean_id_time = cleaned_sub_cate_price_rate_asin_anlz_brand_pt[['reviewerID', 'reviewTime']]
sum_df(clean_id_time)

type <class 'pandas.core.frame.DataFrame'>
shape (6724, 2)


In [21]:
cleaned_review_small = cleaned_sub_cate_price_all[['reviewerID', 'reviewTime','asin']]
sum_df(cleaned_review_small)

type <class 'pandas.core.frame.DataFrame'>
shape (185514, 3)


---
# <font color = blue> Efficient computing for high-dimensional big data for diversity index for asin
---
* reduce the original dataframe with only required low: clean_id_time(=6724 reviewer list and time of the first review on the programmable thermostat), cleaned_sub_cate_price_small)
* DF1: **clean_id_time**(=6724,2) :  reviewer list and time of the first review on the programmable thermostat)
* DF2: **cleaned_review_small**(=185514, 3): cleaned reviews without missing values in price and brand
* option1 : split into cores and multicoreprocess
* option2 : dask dataframe work
* option3 : using more cores with big-memories
---

In [22]:
# adjust for multi_core_process_diversity_index
# multi_core_asin_di(clean_id_time) 
def multi_core_asin_di(df): # df = clean_id_time, -> tun to be  dic = dic_reviewer_time
    
    dic = dictionary_for_reviewer_time(df)
    person  = 0
    em_list =[]
    
    # u_asin_list # = unique_value_list(df,'asin') # 124368 unique list 
       
    for user, time in dic.items():
        print(user, time)
        
        em_df = pd.DataFrame({'reviewerID': [user],'reviewTime':[time]})
            
        n=0
        
        for i in u_asin_list:
        
            em_df['sum_{}'.format(i)] =  len(cleaned_review_small[(cleaned_review_small['reviewerID'] == user) & (cleaned_review_small['reviewTime'] < time) & (cleaned_review_small['asin'] == i)])
        
            if n% 1000 ==0:
                print('first loop for {}'.format(user), n)
            n+=1
        
        
        total = 0
    
        for i in range(2, em_df.shape[1]): #shape[1] column
            total += em_df.iloc[:,i]
            if i% 1000 ==0:
                 print('second loop', i)
        
        p=0.0
        h=0.0
    
        for i in range(2, em_df.shape[1]):
            p = float(em_df.iloc[:,i]/total)
            if  p == 0.0:
                h += 0
            elif p > 0.0: 
                 h += -1*p*np.log(p) # add +1 for -inf case, log is natural logrithm
                
        em_df['asin_diversity'] = h  # this is the category diversity index for reviewer at time t considering t-1 reviews  
        em_df['cum_review_t-1'] = total 
        em_list.append(em_df)
       
        if person% 1000 ==0:
            print(person)
        person+=1
        
    asin_analysis_df = pd.concat(em_list)
    # df = pd.merge(reviewer_time_df,cate_analysis_df, how='left', on='reviewerID')
    return asin_analysis_df    

In [35]:
# Demo for small set
clean_id_time.head(5)

,reviewerID,reviewTime
0,AVHZP1NAWPLD9,2005-03-14
1,ANRXH0BVEAR6B,2005-07-19
2,A2VIFZ0H7XCKP1,2005-10-12
3,A1RPXPCEY07ODW,2005-11-08
4,AQVO6SZI2027K,2005-11-12


In [ ]:
# 6724 x 3 times /24 = days requires, allocating many node as possible 
asin_analysis_df = parallelize(clean_id_time.head(20), multi_core_asin_di)

---
# <font color = blue> Part2 Dask Version
---
* DF1: clean_id_time_dd
* DF2: cleaned_review_small_dd: 
* Ref: https://docs.dask.org/en/latest/install.html (conda install dask)
* Ref; https://datascienceschool.net/view-notebook/2282b75b2a63448087b77269885c27cb/ (Korean)
* Ref: https://towardsdatascience.com/trying-out-dask-dataframes-in-python-for-fast-data-analysis-in-parallel-aa960c18a915 (English)
---

In [42]:
import dask.dataframe as dd
from dask.multiprocessing import get
from dask.diagnostics import ProgressBar # for operation process notation
pbar = ProgressBar()
pbar.register()

---
### change it to pandas dataframe to dask dataframe
---

In [28]:
clean_id_time_dd = dd.from_pandas(clean_id_time, npartitions=1)
type(clean_id_time_dd)

dask.dataframe.core.DataFrame

In [35]:
len(clean_id_time_dd)

[########################################] | 100% Completed |  0.1s


6724

In [29]:
cleaned_review_small_dd = dd.from_pandas(cleaned_review_small, npartitions=1)
type(cleaned_review_small_dd)

dask.dataframe.core.DataFrame

In [36]:
len(cleaned_review_small_dd )

[########################################] | 100% Completed |  0.1s


185514

---
### check functional level operation with dask
---

In [39]:
dictionary_for_reviewer_time(clean_id_time_dd.head(1))

[########################################] | 100% Completed |  0.1s


{'AVHZP1NAWPLD9': '2005-03-14'}

In [41]:
em_df = pd.DataFrame({'reviewerID': ['jikhan'],'reviewTime':['Jan-24']})
em_df = dd.from_pandas(em_df, npartitions=1)   
em_df.compute()

[########################################] | 100% Completed |  0.1s


,reviewerID,reviewTime
0,jikhan,Jan-24


In [48]:
cleaned_review_small.head(2)

,reviewerID,reviewTime,asin
0,A0191512Q2Z9IPUAE2RZ,2013-07-06,B000BOBWBG
1,A0191512Q2Z9IPUAE2RZ,2013-07-06,B00894YWD0


In [51]:
u_asin_list[10]

'0006514006'

In [52]:
user_demo = 'A0191512Q2Z9IPUAE2RZ'
time_demo = '2014-07-06'
asin_demo = 'B00894YWD0' # one of unique asin 
len(cleaned_review_small_dd[(cleaned_review_small_dd['reviewerID'] == user_demo) & (cleaned_review_small_dd['reviewTime'] < time_demo) & (cleaned_review_small_dd['asin'] == asin_demo)])
# all work

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s


1

---
### Dask Version of Multi-Core processing to get diversity index for product level
---

In [ ]:
# DASK version for memory efficiency
# adjust for multi_core_process_diversity_index
# multi_core_asin_di(clean_id_time_dd) + cleaned_review_small_dd
def multi_core_asin_di_dd(df): # df = clean_id_time_dd, -> tun to be  dic = dic_reviewer_time
    
    dic = dictionary_for_reviewer_time(df) # work = turn to dictionarly
    person  = 0
    em_list =[]
    
    # u_asin_list # = unique_value_list(df,'asin') # 124368 unique list 
       
    for user, time in dic.items():
        print(user, time)
        
        em_df = pd.DataFrame({'reviewerID': [user],'reviewTime':[time]})
        em_df = dd.from_pandas(em_df, npartitions=1)    
        n=0
        
        for i in u_asin_list:
        
            em_df['sum_{}'.format(i)] =  len(cleaned_review_small_dd[(cleaned_review_small_dd['reviewerID'] == user) & (cleaned_review_small_dd['reviewTime'] < time) & (cleaned_review_small_dd['asin'] == i)])
        
            if n% 1000 ==0:
                print('first loop for {}'.format(user), n)
            n+=1
        
        
        total = 0
    
        for i in range(2, em_df.shape[1]): #shape[1] column
            total += em_df.iloc[:,i]
            if i% 1000 ==0:
                 print('second loop', i)
        
        p=0.0
        h=0.0
    
        for i in range(2, em_df.shape[1]):
            p = float(em_df.iloc[:,i]/total)
            if  p == 0.0:
                h += 0
            elif p > 0.0: 
                 h += -1*p*np.log(p) # add +1 for -inf case, log is natural logrithm
                
        em_df['asin_diversity'] = h  # this is the category diversity index for reviewer at time t considering t-1 reviews  
        em_df['cum_review_t-1'] = total 
        em_list.append(em_df)
       
        if person% 1000 ==0:
            print(person)
        person+=1
        
    asin_analysis_df = dd.concat(em_list)
    print('the length of asin_anslysis: ', len(asin_analysis_df)
    # df = pd.merge(reviewer_time_df,cate_analysis_df, how='left', on='reviewerID')
    return asin_analysis_df    

In [54]:
len(clean_id_time_dd)

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


6724

In [67]:
# start demo with 24 reviewer = 1/24/2020 1pm 30min,  124368 unique products,
# need to check again with individual level and asin_index is correct or not.
%%time
asin_analysis_df = parallelize(clean_id_time_dd.head(24), multi_core_asin_di)

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
ABA5CXZHS0U9Z 2006-04-11
A2T6GWBGR4XQ3L 2006-03-14
AQVO6SZI2027K 2005-11-12
AA3R95ETMLG1B 2006-08-04
ANRXH0BVEAR6B 2005-07-19
A1V2PDUZVMGRV7 2006-08-03
A1RPXPCEY07ODW 2005-11-08
AYL2RCBAY86ML 2006-08-26
A13SBXZOMP7WS5 2006-01-30
A34OBTZRRZEXFX 2006-10-13
A202DC58JWUAOS 2006-08-12
AVHZP1NAWPLD9 2005-03-14
A1ETHUCBRA7URP 2006-03-29
A1QO7UOWLOYIFV 2006-09-15
A1NJHQ0YZHDYUH 2006-10-24
AB0NVTUKL7D5Z 2006-09-22
A1JR5RLX32ENMM 2006-08-26
A2VIFZ0H7XCKP1 2005-10-12
A1GO68JYWJ5WFJ 2006-03-17
A101OAAMZYWQ3U 2006-09-05
A3VF3S9EH2EGS5 2005-11-30
ARYKQVYDW2PO6 2006-07-25
AUV2X1BK2VHKN 2006-02-25
A23SUOMYEHK0JZ 2006-08-30
first loop for A1JR5RLX32ENMM 0
first loop for ARYKQVYDW2PO6 0
first loop for AYL2RCBAY86ML 0
first loop for A34OBTZRRZEXFX 0
first loop for A1QO7UOWLOYIFV 0
first loop for ABA5CXZHS0U9Z 0
first loop for A13SBXZOMP7WS5 0
first loop for A1GO68JYWJ5WF

Process ForkPoolWorker-71:
Process ForkPoolWorker-62:
Process ForkPoolWorker-64:
Process ForkPoolWorker-63:
Process ForkPoolWorker-55:
Process ForkPoolWorker-49:
Process ForkPoolWorker-52:
Process ForkPoolWorker-58:
Process ForkPoolWorker-51:
Process ForkPoolWorker-56:
Process ForkPoolWorker-54:
Process ForkPoolWorker-50:
Process ForkPoolWorker-53:
Process ForkPoolWorker-69:
Process ForkPoolWorker-61:
Process ForkPoolWorker-67:
Process ForkPoolWorker-70:
Process ForkPoolWorker-66:
Process ForkPoolWorker-68:
Process ForkPoolWorker-57:
Process ForkPoolWorker-59:
Process ForkPoolWorker-60:
Process ForkPoolWorker-65:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-72:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Tr

KeyboardInterrupt: 

  File "/opt/apps/anaconda3/19.10.0/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/apps/anaconda3/19.10.0/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/apps/anaconda3/19.10.0/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/apps/anaconda3/19.10.0/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/apps/anaconda3/19.10.0/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/apps/anaconda3/19.10.0/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (m

# Interpretation

In [68]:
# Reviewer / 24 hours
6724/24

280.1666666666667

In [71]:
# requried days for computation
280 * 3 /24
# 35 days requires for this computation

35.0

* To get the product level diversity in all reviewers requires more than 1 months stop it in here.
* The solution may using gpu, larger memory in HPC; however, I already got the category level diveristy so that stop it here. it is just for the general public